In [1]:
import os
import sys
import numpy as np
import pandas as pd
import openmatrix as omx

In [2]:
os.chdir('../data/TransportationNetworks')
root = os.getcwd()
root

'c:\\Projects\\TrafficAssignmentApproaches\\data\\TransportationNetworks'

In [3]:
folders = [x for x in os.listdir(root)[1:] if os.path.isdir(os.path.join(root, x))]
folders

['.git',
 'Anaheim',
 'Austin',
 'Barcelona',
 'Berlin-Center',
 'Berlin-Friedrichshain',
 'Berlin-Mitte-Center',
 'Berlin-Mitte-Prenzlauerberg-Friedrichshain-Center',
 'Berlin-Prenzlauerberg-Center',
 'Berlin-Tiergarten',
 'Birmingham-England',
 'Braess-Example',
 'chicago-regional',
 'Chicago-Sketch',
 'Eastern-Massachusetts',
 'GoldCoast',
 'Hessen-Asymmetric',
 'Munich',
 'Philadelphia',
 'SiouxFalls',
 'Sydney',
 'SymmetricaTestCase',
 'Terrassa-Asymmetric',
 'Winnipeg',
 'Winnipeg-Asymmetric',
 '_scripts']

In [4]:
# Function to import OMX matrices
def import_matrix(matfile):
    f = open(matfile, 'r')
    all_rows = f.read()
    blocks = all_rows.split('Origin')[1:]
    matrix = {}
    for k in range(len(blocks)):
        orig = blocks[k].split('\n')
        dests = orig[1:]
        orig=int(orig[0])

        d = [eval('{'+a.replace(';',',').replace(' ','') +'}') for a in dests]
        destinations = {}
        for i in d:
            destinations = {**destinations, **i}
        matrix[orig] = destinations
    zones = max(matrix.keys())
    mat = np.zeros((zones, zones))
    for i in range(zones):
        for j in range(zones):
            mat[i, j] = matrix.get(i+1,{}).get(j+1,0)
            
    return mat

In [7]:
import_matrix(os.path.join(root, 'SiouxFalls', 'SiouxFalls' + '_trips.tntp'))

array([[   0.,  100.,  100.,  500.,  200.,  300.,  500.,  800.,  500.,
        1300.,  500.,  200.,  500.,  300.,  500.,  500.,  400.,  100.,
         300.,  300.,  100.,  400.,  300.,  100.],
       [ 100.,    0.,  100.,  200.,  100.,  400.,  200.,  400.,  200.,
         600.,  200.,  100.,  300.,  100.,  100.,  400.,  200.,    0.,
         100.,  100.,    0.,  100.,    0.,    0.],
       [ 100.,  100.,    0.,  200.,  100.,  300.,  100.,  200.,  100.,
         300.,  300.,  200.,  100.,  100.,  100.,  200.,  100.,    0.,
           0.,    0.,    0.,  100.,  100.,    0.],
       [ 500.,  200.,  200.,    0.,  500.,  400.,  400.,  700.,  700.,
        1200., 1400.,  600.,  600.,  500.,  500.,  800.,  500.,  100.,
         200.,  300.,  200.,  400.,  500.,  200.],
       [ 200.,  100.,  100.,  500.,    0.,  200.,  200.,  500.,  800.,
        1000.,  500.,  200.,  200.,  100.,  200.,  500.,  200.,    0.,
         100.,  100.,  100.,  200.,  100.,    0.],
       [ 300.,  400.,  300.,  400.,

In [5]:
# Importing the networks into a Pandas dataframe consists of a single line of code
# but we can also make sure all headers are lower case and without trailing spaces
def import_net(f):
    netfile = os.path.join(root, f, f + '_net.tntp')
    net = pd.read_csv(netfile, skiprows=8, sep='\t')

    trimmed= [s.strip().lower() for s in net.columns]
    net.columns = trimmed

    # And drop the silly first andlast columns
    net.drop(['~', ';'], axis=1, inplace=True)
    return net

In [8]:
import_net('Braess')

,init_node,term_node,capacity,length,free_flow_time,b,power,speed,toll,link_type
0,1,3,1,100,1.000000e-08,1.000000e+09,1,0,0,1
1,1,4,1,100,5.000000e+01,2.000000e-02,1,0,0,1
2,3,2,1,100,5.000000e+01,2.000000e-02,1,0,0,1
3,3,4,1,100,1.000000e+01,1.000000e-01,1,0,0,1
4,4,2,1,100,1.000000e-08,1.000000e+09,1,0,0,1;


In [9]:
def process_dataset(net_name): 
    mat = import_matrix(os.path.join(root, net_name, net_name + '_trips.tntp'))
    net = import_net(net_name)
    number_of_zones = mat.shape[0]
    number_of_nodes = net[['init_node', 'term_node']].max().max()
    number_of_links = net.shape[0]
    with open(os.path.join(root, net_name,  net_name + '_net.csv'), 'w') as f:
        f.write(f"# NUMBER OF ZONES:{number_of_zones},")
        f.write(f"NUMBER OF NODES:{number_of_nodes},")
        f.write(f"NUMBER OF LINKS:{number_of_links}")
        f.write("\n")

        net.to_csv(f, index=False)
    
    with open(os.path.join(root, net_name,  net_name + '_trips.csv'), 'w') as f:
        f.write(f"# NUMBER OF ZONES:{number_of_zones}")
        f.write("\n") 
        
        np.savetxt(f, mat, delimiter=',', fmt='%d')

In [10]:
process_dataset('Braess')